# Dependency Agent Testing

This is an agent that will attempt to solve maven compiler errors using tools and an agentic approach

We use LangGraph for implementing this.

We create a network of agents with smart edges to determine what to do


# Fix Generation Agent

We are using a few hardcoded examples of analysis issues to generate fixes.

Before proceeding, make sure you are using Kai venv to run cells in this notebook.

We need to install `langgraph` module, run following cell:

In [ ]:
%load_ext dotenv
%dotenv
%pip install langgraph

In [ ]:
from kai.reactive_codeplanner.agentic.tools.tools import GetAllAvailableDependencies
from pathlib import Path

all_deps_tool = GetAllAvailableDependencies()

print(all_deps_tool.get_input_schema().model_json_schema())
print(all_deps_tool.tool_call_schema.model_json_schema())
print(all_deps_tool.get_output_jsonschema())


## Validating tool usage with different providers via Model Provider

This section validates that we are able to use tools with different Model Providers we support.

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState 
from langgraph.prebuilt import ToolNode
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage
from kai.llm_interfacing.model_provider import ModelProvider
from kai.reactive_codeplanner.agentic.schemas.maven_compiler.schema import InitState
from langgraph.types import Command


def validate(model: ModelProvider, **kwargs):

    def should_continue(state: InitState):
        messages = state.messages
        last_message = messages[-1]
        print(f"should conintue last message {last_message}")
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            print("returning tools") 
            return "tools"
        return END


    def call_model(state: InitState):
        if state.dependencies: 
            print(f"finished --> {state.dependencies}")
            return
        messages = state.messages
        response = model.bind_tools([GetAllAvailableDependencies()]).invoke(messages)
        state.messages.append(response)
        return state.model_dump()


    graph_builder = StateGraph(InitState)
    graph_builder.add_node("tools", ToolNode([GetAllAvailableDependencies()]))
    graph_builder.add_node("agent", call_model)
    graph_builder.add_edge(START, "agent")
    graph_builder.add_conditional_edges("agent", should_continue, ["tools", END])
    graph_builder.add_edge("tools", "agent")

    app = graph_builder.compile()

    for r in app.stream({"messages": [HumanMessage("please list all the dependencies for the given java project")], "file_path": "/Users/shurley/repos/kai/kai/example/coolstore/pom.xml", "task": {}, "background": "", "dependencies": []}, stream_mode="values"):
        print(r)
    


Depending on which model you test this with, you need to set up your API keys for the model before running the "validation" cell. Every cell has a comment at top about the environment variable expected.

You will set the keys in `.env` file at the project root and run the following cell for changes to take effect.

### Results

|   Kai Model Provider   |      Model       |  Works?  |
| ---------------------- | ---------------- | -------- |
| ChatOpenAI             | gpt-3.5-turbo    |  &check; |
| ChatOpenAI (RH Maas)   | granite-8b       |  &check; |
| ChatOpenAI (RH Maas)   | llama-7b         |  &check; |
| ChatBedrock            | llama-70b-v1     |  &cross; |
| ChatBedrock            | llama-70b-v2     |  &cross; |
| ChatBedrock            | claude-3-sonnet  |  &check; |
| ChatGoogleGenAI        | gemini-2.0-flash |  &check; |
| ChatDeepSeek           | deepseek-chat    |  &check; |

Following model providers are not tested yet:

* ChatOllama
* ChatAzureOpenAI

In [ ]:
%dotenv

In [ ]:
## Test with granite-8b on RH maas via ChatOpenAI
## Env vars needed:
## PARASOL_GRANITE_KEY - api key
## PARASON_GRANITE_API - base url (usually ends with /v1)

from kai.llm_interfacing.model_provider import ChatOpenAI

validate(ChatOpenAI(
    api_key=os.environ["PARASOL_GRANITE_KEY"],
    base_url=os.environ["PARASOL_GRANITE_API"],
    model_name="granite-3-8b-instruct",
))

In [ ]:
from kai.llm_interfacing.model_provider import ChatOpenAI
import os
validate(ChatOpenAI(
    api_key=os.environ["PARASOL_LLAMA_KEY"],
    base_url=os.environ["PARASOL_LLAMA_API"],
    model_name="meta-llama/Llama-3.1-8B-Instruct",
))

In [ ]:
## Test with gpt-3.5-turbo via ChatOpenAI
## Env vars needed:
## OPENAI_API_KEY - api key

validate(ChatOpenAI(
    model="gpt-3.5-turbo"
))

In [ ]:
## Test with gemini-2.0-flash via GoogleGenAI
## Env vars needed:
## GEMINI_API_KEY - api key

from kai.llm_interfacing.model_provider import ChatGoogleGenerativeAI

validate(ChatGoogleGenerativeAI(
    api_key=os.environ["GEMINI_API_KEY"],
    model="gemini-2.0-flash",
))

In [ ]:
## Test with llama-70b via Bedrock
## Env vars needed:
## AWS_ACCESS_KEY_ID
## AWS_SECRET_ACCESS_KEY

from kai.llm_interfacing.model_provider import ChatBedrock

validate(ChatBedrock(
    model="us.meta.llama3-1-70b-instruct-v1:0"
))

In [ ]:
validate(ChatBedrock(
    model="us.meta.llama3-2-1b-instruct-v1:0",
))

In [ ]:
validate(ChatBedrock(
    model="us.meta.llama3-3-70b-instruct-v1:0"
))

In [ ]:
validate(ChatBedrock(
    model="us.anthropic.claude-3-sonnet-20240229-v1:0",
))

In [ ]:
# Env vars needed:
# DEEPSEEK_API_KEY

from kai.llm_interfacing.model_provider import ChatDeepSeek

validate(model=ChatDeepSeek(
    api_key=os.environ["DEEPSEEK_API_KEY"],
    model="deepseek-chat",
), interrupt_after=["tools"])

### Using tools with models that do not support tool calling

In this section, we experiment creating our own _LangGraph ToolNode_ and custom logic in the _Conditional Edge_ to enable models that don't support tools to call given tools. Note that we saw examples of models in previous sections that do not support tools out of the box. It is absolutely essential that we are able to use tools with any model if we have to use LangGraph approach for agents. We validate whether its possible to do that in this section.

In [ ]:
## we use meta-llama-70b provided via Bedrock in this section

import json
from typing import Optional, TypedDict
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, SystemMessage, ToolMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain.tools.render import render_text_description
from kai.llm_interfacing.model_provider import ChatBedrock

def _print_state(state: MessagesState):
    print("..."*15)
    for idx, message in enumerate(state["messages"]):
        print(f"{idx}: {message.content.splitlines()[0] if message.content else message.content}")
    print("..."*15)

@tool
def get_temperature(city: Annotated[str, "Name of the city"]):
    """Returns current temperature in given city"""
    return f"Current temperature in {city} is 80 degrees."

# define the tool node
tool_node = ToolNode(tools=[get_temperature])

# define the model provider
model_provider = ChatBedrock(
    model="us.meta.llama3-3-70b-instruct-v1:0"
)

def extract_json_from_text(text):
    """Extracts JSON code block from a multiline string."""
    pattern = r"```(json)?\s*(\{.*?\})\s*```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        json_str = match.group(2)
        try:
            return json.loads(json_str)  # Convert to Python dict
        except json.JSONDecodeError:
            return None  # Invalid JSON format

    return None

# define the agent
def agent_call(state: MessagesState):
    # explain the tools here
    sys_msg = f"""You are an intelligent assistant who can help users with their weather related queries. 
You may not know answers to all questions as some of the answers may depend on real time weather data. However, you are given tools you can use to access that data.
Here is the schema of tools you are given:

{render_text_description([get_temperature])}

If you do need to call a tool, respond with a JSON object containing only two keys - tool_name and args. `tool_name` should be the name of the tool to call and `args` should be nested JSON containing the arguments to pass to the function in key value format.
Make sure you always use ``` at the start and end of the JSON block to clearly separate it from text.
If you have reached the answer, respond with text and add string 'FINAL ANSWER' to the beginning.
"""
    # insert tool description as sys message
    if "messages" not in state or not state["messages"] or \
        not isinstance(state["messages"][0], SystemMessage):
        state["messages"].insert(0, SystemMessage(content=sys_msg))
    
    # ToolMessage is a special type of message that contains the tool response.
    # our model doesn't understand that. we change it to a human message instead.
    if isinstance(state["messages"][-1], ToolMessage):
        tool_output = state["messages"][-1]
        state["messages"][-1] = HumanMessage(
            content=f"The output of tool {tool_output.name} is - {tool_output.content}"
        )
    
    response = model_provider.invoke(state["messages"])

    # our model does not set tool_calls on the response
    # we parse the model response ourselves and convert
    # it into an AIMessage with tool_calls set
    if "tool_name" in response.content:
        agent_response = extract_json_from_text(response.content)
        if not agent_response:
            return { "messages": [AIMessage(content="RETRY")] }
        response = AIMessage(
            content="",
            tool_calls=[{
                "name": agent_response.get("tool_name"),
                "args": agent_response.get("args"),
                "type": "tool_call",
                "id": "tool_call_id_1",
            }],
        )
    return { "messages": [response] }

# this is the conditional edge to parse and
# route the llm response to a tool
def should_call_tool(state: MessagesState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools"
    if not last_message.content or "RETRY" in last_message.content:
        return "agent"
    return END
    
workflow = StateGraph(MessagesState)

workflow.add_node("agent", agent_call)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_call_tool, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile()

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass
    

In [ ]:
for chunk in app.stream(
    {"messages": [("human", "what's the weather in New York?")]}, stream_mode="values"
):
    print(chunk["messages"][-1].pretty_repr())